In [2]:
# Optimal pandas import
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

pd.options.display.max_colwidth = -1
pd.options.display.max_rows = 999

In [108]:
COD_DEP

,CODIGO DEPARTAMENTO,CODIGO PROVINCIA,DEPARTAMENTO,dpto_merger_str
0,1,1,COMUNA 1,comuna 1
1,2,1,COMUNA 2,comuna 2
2,3,1,COMUNA 3,comuna 3
3,4,1,COMUNA 4,comuna 4
4,5,1,COMUNA 5,comuna 5
5,6,1,COMUNA 6,comuna 6
6,7,1,COMUNA 7,comuna 7
7,8,1,COMUNA 8,comuna 8
8,9,1,COMUNA 9,comuna 9
9,10,1,COMUNA 10,comuna 10


In [162]:
# Cargar info distritos.
## Necesitamos arreglar los nombres porque tenemor que mergear en base a eso.
## CNE DEBERIA USAR CODIGOS ESTANDARIZADOS DE PARTAMENTOS COMO CENSO INDEC.

In [196]:
from unidecode import unidecode

COD_PROV = pd.read_csv('./codigos/COD_PROVINCIAS.csv', ';', encoding = 'latin-1').rename(columns = {'NOMBRE': 'PROVINCIA'})
COD_DEP = pd.read_csv('./codigos/COD_DEPARTAMENTOS.csv', ';', encoding = 'latin-1').rename(columns = {'NOMBRE': 'DEPARTAMENTO'})
#insertar 0 en nombres de comuna, ej. Comuna 05
COD_DEP['dpto_merger_str'] = COD_DEP['DEPARTAMENTO'].str.replace('12 ', 'Doce'
                                            ).str.replace('1° ', 'Primero'
                                            ).str.replace('2 ', 'Dos '
                                            ).str.replace('25', 'Veinticinco'
                                            ).str.replace('9 ', 'Nueve ')

COD_DEP['dpto_merger_str'] = [unidecode(latin_str).lower() for latin_str in COD_DEP['DEPARTAMENTO']]
COD_DEP.loc[:8, 'dpto_merger_str'] = [s[:7] + '0' + s[7:] for s in COD_DEP.loc[:8]['dpto_merger_str'].values]

# Fix differences for BUENOS AIRES dptos
COD_DEP['dpto_merger_str'] = COD_DEP['dpto_merger_str'].str.replace('matanza', 'la matanza'
                                            ).str.replace('j. c. paz', 'jose c. paz'
                                            ).str.replace('roque saenz pena', 'presidente roque saenz pena '
                                            ).str.replace('coronel rosales', 'coronel de marina l. rosales'
                                            ).str.replace('general lamadrid', 'general la madrid'
                                            ).str.replace('las heras', 'general las heras'
                                            ).str.replace('general madariaga', 'general juan madariaga'
                                            ).str.replace('gonzales chaves', 'adolfo gonzales chaves'
                                            ).str.replace('f. ameghino', 'florentino ameghino'
                                            ).str.replace('juarez', 'benito juarez'
                                            ).str.replace('arrecifes', 'bartolome mitre'
                                            ).str.replace('lezama', 'chascomus')
geo_ref = pd.read_csv('./../../Dropbox/PBAshared/censo_geo/radio_ref.csv')
geo_ref['dpto_merger_str'] = geo_ref['NOMDPTO'].str.replace('12 ', 'Doce'
                                            ).str.replace('1° ', 'Primero'
                                            ).str.replace('2 ', 'Dos '
                                            ).str.replace('25', 'Veinticinco'
                                            ).str.replace('9 ', 'Nueve ')

geo_ref['dpto_merger_str'] = geo_ref['dpto_merger_str'].str.strip()
geo_ref['dpto_merger_str'] = [unidecode(latin_str).lower() for latin_str in geo_ref['dpto_merger_str']]
# geo_ref['dpto_merger_str'] = geo_ref['dpto_merger_str'].str.replace('arrecifes', 'bartolome mitre')

geo_ref.head()

# Las cantidades matchean. Bien!
# Tambien los codigos de provincia.
# geo_ref.groupby('PROV_REF_ID')['DPTO'].nunique()
# COD_DEP.groupby('CODIGO PROVINCIA')['DEPARTAMENTO'].nunique()

merged = COD_DEP.merge(geo_ref[['PROV_REF_ID','PROV','DPTO', 'dpto_merger_str']].drop_duplicates(),
              left_on = ['CODIGO PROVINCIA', 'dpto_merger_str'], right_on = ['PROV_REF_ID', 'dpto_merger_str'], how = 'left')
merged = merged.dropna() # perdemos dptos no matcheados del interior

IDcols = ['PROV_REF_ID', 'PROV', 'DPTO']
merged[IDcols] = merged[IDcols].applymap(int).applymap(str)


# Las provincias completas son CABA y GBA. Resto del pais hay como 30/350 dptos que no matchean
dpto_ref_BA = merged.loc[merged.PROV_REF_ID.isin(['1', '2'])].set_index('DEPARTAMENTO')[['DPTO', 'dpto_merger_str']]
display(dpto_ref_BA.sample(10))
# Ya esta todo linkeado con DPTOS geo, se pueden hacer mapas.

,DPTO,dpto_merger_str
DEPARTAMENTO,,
BOLIVAR,6105,bolivar
SAN ANTONIO DE ARECO,6735,san antonio de areco
CAMPANA,6126,campana
SALTO,6714,salto
F. AMEGHINO,6277,florentino ameghino
MATANZA,6427,la matanza
COMUNA 15,2015,comuna 15
BERISSO,6098,berisso
COMUNA 6,2006,comuna 06


# Data Electoral

In [154]:
#Load data
primera_vuelta_2015 = pd.concat([pd.read_csv('./escrutinio/2015/FMESDN_0101.csv', ';'),
                            pd.read_csv('./escrutinio/2015/FMESDN_0202.csv', ';'),
                            pd.read_csv('./escrutinio/2015/FMESDN_0313.csv', ';'),
                            pd.read_csv('./escrutinio/2015/FMESPR_1424.csv', ';')])
# primera_vuelta_2015['ELECCION'] = 'PV_2015'

COD_PARTIDOS = pd.read_csv('./codigos/COD_PARTIDOS_2015.csv', ';', encoding = 'latin-1')[[u'Codigo Partido', u'Denominacion']]
COD_PARTIDOS['Denominacion'] = COD_PARTIDOS['Denominacion'].str.replace('CAMBIEMOS BUENOS AIRES', 'ALIANZA CAMBIEMOS').str.strip()

primera_vuelta_2015 = primera_vuelta_2015.merge(COD_PARTIDOS, left_on = u'CODIGO VOTOS', right_on = u'Codigo Partido')
# # primera_vuelta_2015.groupby([u'Codigo Partido', u'Denominacion']).sum().sort_values(by = 'VOTOS')[['VOTOS']].tail(10)

primera_vuelta_2015 = primera_vuelta_2015.merge(COD_PROV).merge(COD_DEP, on = ['CODIGO DEPARTAMENTO', u'CODIGO PROVINCIA'])
# #fix
primera_vuelta_2015.columns = primera_vuelta_2015.columns.str.strip()
primera_vuelta_2015['CODIGO CIRCUITO'] = primera_vuelta_2015['CODIGO CIRCUITO'].astype(str).str.strip().str.zfill(4)

In [155]:
primera_vuelta_2015.nunique()

CODIGO PROVINCIA       24  
CODIGO DEPARTAMENTO    135 
CODIGO CIRCUITO        1938
CODIGO MESA            8597
AMBITO                 1   
CODIGO VOTOS           28  
VOTOS                  471 
Codigo Partido         28  
Denominacion           24  
PROVINCIA              24  
DEPARTAMENTO           445 
dpto_merger_str        444 
dtype: int64

In [160]:
merged.groupby('CODIGO PROVINCIA')['DPTO'].nunique()

CODIGO PROVINCIA
1.0     15 
2.0     134
3.0     16 
4.0     23 
5.0     25 
6.0     17 
7.0     14 
8.0     17 
9.0     9  
10.0    15 
11.0    21 
12.0    13 
13.0    16 
14.0    13 
15.0    16 
16.0    10 
17.0    22 
18.0    18 
19.0    5  
20.0    7  
21.0    19 
22.0    25 
23.0    15 
24.0    2  
Name: DPTO, dtype: int64

In [161]:
merged.loc[merged.DPTO.isna() | merged.DEPARTAMENTO.isna()].sort_values(by = ['PROV_REF_ID', 'DPTO'])

,CODIGO DEPARTAMENTO,CODIGO PROVINCIA,DEPARTAMENTO,dpto_merger_str,PROV_REF_ID,PROV,DPTO
527,NaN,NaN,NaN,juarez celman,4.0,14.0,14056.0
528,NaN,NaN,NaN,marcos juarez,4.0,14.0,14063.0
529,NaN,NaN,NaN,presidente roque saenz pena,4.0,14.0,14084.0
530,NaN,NaN,NaN,docede octubre,6.0,22.0,22036.0
531,NaN,NaN,NaN,dos de abril,6.0,22.0,22039.0
532,NaN,NaN,NaN,fray justo santa maria de oro,6.0,22.0,22043.0
533,NaN,NaN,NaN,libertador general san martin,6.0,22.0,22084.0
534,NaN,NaN,NaN,mayor luis j. fontana,6.0,22.0,22098.0
535,NaN,NaN,NaN,nueve de julio,6.0,22.0,22105.0
536,NaN,NaN,NaN,presidencia de la plaza,6.0,22.0,22119.0


In [121]:
merged.loc[merged.DEPARTAMENTO.isna()].sort_values(by = ['PROV_REF_ID', 'DPTO'])

,CODIGO DEPARTAMENTO,CODIGO PROVINCIA,DEPARTAMENTO,dpto_merger_str,PROV_REF_ID,PROV,DPTO
527,NaN,NaN,NaN,comuna 01,1.0,2.0,2001.0
528,NaN,NaN,NaN,comuna 02,1.0,2.0,2002.0
529,NaN,NaN,NaN,comuna 03,1.0,2.0,2003.0
530,NaN,NaN,NaN,comuna 04,1.0,2.0,2004.0
531,NaN,NaN,NaN,comuna 05,1.0,2.0,2005.0
532,NaN,NaN,NaN,comuna 06,1.0,2.0,2006.0
533,NaN,NaN,NaN,comuna 07,1.0,2.0,2007.0
534,NaN,NaN,NaN,comuna 08,1.0,2.0,2008.0
535,NaN,NaN,NaN,comuna 09,1.0,2.0,2009.0
538,NaN,NaN,NaN,adolfo gonzales chaves,2.0,6.0,6014.0
